In [1]:
import sys
import numpy as np
import h5py
import re

In [68]:
#BED file
filename = '/run/media/jmorlot/add51d2e-cd8e-48fb-93dd-4767883364a6/ChIA-Pet/interaction_GSM1872887_GM12878_RNAPII_PET_clusters.txt'
fbed = open(filename,'r')

chrmlentot = np.array([249250621,243199373,198022430,191154276,180915260,171115067,159138663,146364022,141213431,135534747,135006516,133851895,115169878,107349540,102531392,90354753,81195210,78077248,59128983,63025520,48129895,51304566])
res = 200
chrmlentotbin = np.cumsum(np.floor(chrmlentot/res))
chrmlentotbin = np.insert(chrmlentotbin, 0, 0)
S = int(np.sum(np.floor(chrmlentot/res)))

vect = np.zeros((S,1))

In [67]:
fbed.close()

In [3]:
header = fbed.readline()

In [7]:
h1 = (header.split('\n')[0]).split('\t')
h2 = h1[3].split(':')
h3 = h2[1].split('-')
h4 = h3[1].split(',')
print h1,h2,h3,h4


['chr1', '567503', '567654', 'chrM:15126-15749,4', '124953', '.'] ['chrM', '15126-15749,4'] ['15126', '15749,4'] ['15749', '4']


In [29]:
print posmax

13317787


In [69]:
from scipy.sparse import coo_matrix

#Estimate list size
L=0
for line in fbed:
    
    h1 = (line.split('\n')[0]).split('\t')
    h2 = h1[3].split(':')
    h3 = h2[1].split('-')
    h4 = h3[1].split(',')
    
    
    chrm1 = h1[0].split('chr')[1]
    chrm2 = h2[0].split('chr')[1]
    
    pos11 = int(h1[1])
    pos12 = int(h1[2])
    
    pos21 = int(h3[0])
    pos22 = int(h4[0])
    
    value = int(h4[1])
    
    if re.search(r'[0-9]', chrm1) and re.search(r'[0-9]', chrm2):
        
        chrm1 = int(chrm1)
        chrm2 = int(chrm2)
        
        if chrm1==chrm2:
            pos11 = int(chrmlentotbin[chrm1-1]+ np.floor(int(pos11)/res))
            pos12 = int(chrmlentotbin[chrm1-1]+ np.floor(int(pos12)/res))
            pos21 = int(chrmlentotbin[chrm2-1]+ np.floor(int(pos21)/res))
            pos22 = int(chrmlentotbin[chrm2-1]+ np.floor(int(pos22)/res))
            
            pos1 = range(pos11,pos12)
            pos2 = range(pos21,pos22)
            
            pos1R = [p for p in pos1 for i in range(len(pos2))]
            pos2R = [p for i in range(len(pos1)) for p in pos2 ]
            valueR = [value for v in range(len(pos1R))]
            
            L = L + len(pos1R)
        

#Preallocating and Filling the vectors
fbed.close()
fbed = open(filename,'r')      

I=np.zeros(L)
J=np.zeros(L)
V=np.zeros(L)
l=0

for line in fbed:
    
    h1 = (line.split('\n')[0]).split('\t')
    h2 = h1[3].split(':')
    h3 = h2[1].split('-')
    h4 = h3[1].split(',')
    
    
    chrm1 = h1[0].split('chr')[1]
    chrm2 = h2[0].split('chr')[1]
    
    pos11 = int(h1[1])
    pos12 = int(h1[2])
    
    pos21 = int(h3[0])
    pos22 = int(h4[0])
    
    value = int(h4[1])
    
    if re.search(r'[0-9]', chrm1) and re.search(r'[0-9]', chrm2):
        
        chrm1 = int(chrm1)
        chrm2 = int(chrm2)
        
        if chrm1==chrm2:
            pos11 = int(chrmlentotbin[chrm1-1]+ np.floor(int(pos11)/res))
            pos12 = int(chrmlentotbin[chrm1-1]+ np.floor(int(pos12)/res))
            pos21 = int(chrmlentotbin[chrm2-1]+ np.floor(int(pos21)/res))
            pos22 = int(chrmlentotbin[chrm2-1]+ np.floor(int(pos22)/res))
            
            pos1 = range(pos11,pos12)
            pos2 = range(pos21,pos22)
            
            pos1R = [p for p in pos1 for i in range(len(pos2))]
            pos2R = [p for i in range(len(pos1)) for p in pos2 ]
            valueR = [value for v in range(len(pos1R))]
            
            li = len(pos1R)
            I[l:l+li] = pos1R
            J[l:l+li] = pos2R
            V[l:l+li] = valueR
            
            l = l + li

matrix = coo_matrix((V, (I, J)), shape=(S,S))
            
#             posmin = np.min([pos11,pos12,pos21,pos22])
#             posmax = np.max([pos11,pos12,pos21,pos22])

#             vect[posmin:posmax] = vect[posmin:posmax] + value
        
fbed.close()

In [79]:
matrix = coo_matrix((V, (I, J)), shape=(S,S))

In [71]:
matrix = matrix + matrix.T

In [80]:
fh5.close()
fh5 = h5py.File(filename.split('.txt')[0] + '.hdf5', "w")
fh5['data'] = matrix.data
fh5['I'] = matrix.row
fh5['J'] = matrix.col
fh5.close()

In [64]:
m=0
for line in fbed:
#     if re.search(r'chr(\d+)', line):
    values = (line.split('\n')[0]).split('\t')
    if re.search(r'[0-9]', values[0]):
#         chrm = int((values[0]).split('chr')[1])
        chrm = int(values[0])
        pos1 = int(chrmlentotbin[chrm-1]+ np.floor(int(values[1])/res))
        pos2 = int(chrmlentotbin[chrm-1]+ np.floor(int(values[2])/res))
        vect[range(pos1,pos2)] = m%100 + 1
        m=m+1
        
fbed.close()

In [13]:
chrmlentot = np.array([249250621,243199373,198022430,191154276,180915260,171115067,159138663,146364022,141213431,135534747,135006516,133851895,115169878,107349540,102531392,90354753,81195210,78077248,59128983,63025520,48129895,51304566])
res = 200
chrmlentotbin = np.cumsum(np.floor(chrmlentot/res))
chrmlentotbin = np.insert(chrmlentotbin, 0, 0)
S = int(np.sum(np.floor(chrmlentot/res)))

pos1 = []
pos2 = []
vect = []

chrm = 1
filename = '/run/media/jmorlot/storage/Données_Lieberman/GM12878_combined/1kb_resolution_intrachromosomal/chr1/MAPQGE30/chr{0}_1kb.RAWobserved'.format(chrm)
fbed = open(filename,'r')

for line in fbed:
    values = (line.split('\n')[0]).split('\t')
    pos1.append(int(np.floor(int(values[0])/res)))
    pos2.append(int(np.floor(int(values[1])/res)))
    vect.append(int(values[2].split('.')[0]))

print pos1.shape,pos2.shape,vect.shape

AttributeError: 'list' object has no attribute 'shape'

In [14]:
for line in fbed:
    #1st part
    values = (line.split('\n')[0]).split('\t')
    if values[0].isdigit() and values[3].isdigit():
        chrm = int(values[0])
        pos1 = int(chrmlentotbin[chrm-1]+ np.floor(int(values[1])/res))
        pos2 = int(chrmlentotbin[chrm-1]+ np.floor(int(values[2])/res))
        vect[range(pos1,pos2)] = vect[range(pos1,pos2)] + 1
        #2nd part
        chrm = int(values[3])
        pos1 = int(chrmlentotbin[chrm-1]+ np.floor(int(values[4])/res))
        pos2 = int(chrmlentotbin[chrm-1]+ np.floor(int(values[5])/res))
        vect[range(pos1,pos2)] = vect[range(pos1,pos2)] + 1

In [14]:
for line in fbed:
    if re.search(r'chr(\d+)', line):
        values = (line.split('\n')[0]).split('\t')
        chrm = int((values[0]).split('chr')[1])
        pos1 = int(chrmlentotbin[chrm-1]+ np.floor(int(values[1])/res))
        pos2 = int(chrmlentotbin[chrm-1]+ np.floor(int(values[2])/res))
        if re.search(r'A.', values[3]):
            vect[range(pos1,pos2)] = vect[range(pos1,pos2)] + 1
        if re.search(r'A.', values[3]):
            vect[range(pos1,pos2)] = vect[range(pos1,pos2)] + 1

In [75]:
# fbed.close()

fh5 = h5py.File(filename.split('.txt')[0] + '.hdf5', "w")
fh5['data'] = vect
fh5.close()

TypeError: Object dtype dtype('O') has no native HDF5 equivalent

In [ ]:
fbed.close()
V = np.hstack((pos1,pos2,vect))
fh5 = h5py.File(filename.split('.narrowPeak')[0] + '.hdf5', "w")
fh5['data'] = vect
fh5.close()

In [22]:
print re.search(r'A.', 'A3')

In [6]:
filename.split('.')[0:-1]

['/users/invites/jmorlot/T\xc3\xa9l\xc3\xa9chargements/GSM935611_hg19_wgEncodeSydhTfbsGm12878Ctcfsc15914c20StdPk']